In [28]:
import sys
!{sys.executable} -m pip install shap



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: C:\Users\Hari\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [30]:
data = pd.read_csv('WA_Fn-UseC_-Marketing-Customer-Value-Analysis.csv')
user_info = data[['Customer Lifetime Value', 'Income', 'Total Claim Amount', 'Marital Status', 'Vehicle Class', 'Education']].copy().dropna()


In [31]:
marital_encoder = LabelEncoder()
vehicle_encoder = LabelEncoder()
education_encoder = LabelEncoder()
user_info['Marital Status'] = marital_encoder.fit_transform(user_info['Marital Status'])
user_info['Vehicle Class'] = vehicle_encoder.fit_transform(user_info['Vehicle Class'])
user_info['Education'] = education_encoder.fit_transform(user_info['Education'])

In [32]:
num_scaler = StandardScaler()
numeric_cols = ['Customer Lifetime Value', 'Income', 'Total Claim Amount']
user_info[numeric_cols] = num_scaler.fit_transform(user_info[numeric_cols])

In [33]:
policies = pd.DataFrame({
    'policy_name': ['Term Life', 'Health Plan', 'Car Insurance', 'Whole Life', 
                    'Term Insurance 1', 'Term Insurance 2', 'Car Insurance 1', 'Car Insurance 2'],
    'monthly_cost': [500, 800, 600, 1000, 400, 700, 450, 750],
    'target_value': [0.5, 0.0, -0.5, 1.0, 0.4, 0.6, -0.6, -0.4],
    'target_income': [0.5, 0.0, -0.5, 1.0, 0.3, 0.7, -0.7, -0.3],
    'target_claims': [0.0, 0.5, 1.0, -0.5, 0.1, -0.1, 0.9, 1.1],
    'target_marital': [1, 1, 0, 1, 1, 1, 0, 0],
    'target_vehicle': [2, 0, 3, 1, 2, 1, 3, 4],
    'target_education': [1, 2, 0, 1, 1, 2, 0, 1]
})


In [34]:
customer_features = user_info[['Customer Lifetime Value', 'Income', 'Total Claim Amount', 'Marital Status', 'Vehicle Class', 'Education']].values
policy_features = policies[['target_value', 'target_income', 'target_claims', 'target_marital', 'target_vehicle', 'target_education']].values
similarity_scores = cosine_similarity(customer_features, policy_features)

train_data = []
train_labels = []
group_sizes = []
for i in range(len(user_info)):
    for j in range(len(policies)):
        train_data.append(np.concatenate([user_info.iloc[i].values, policy_features[j]]))
        train_labels.append(similarity_scores[i][j])
    group_sizes.append(len(policies))

In [35]:
dtrain = xgb.DMatrix(np.array(train_data), label=np.array(train_labels))
dtrain.set_group(group_sizes)
xgb_model = xgb.train({'objective': 'rank:pairwise', 'eta': 0.1, 'max_depth': 6, 'eval_metric': 'ndcg'}, dtrain, num_boost_round=100)

In [36]:
def get_user_details():
    print("Enter your info for a policy recommendation:")
    income = float(input("Annual Income (e.g., 50000): "))
    marital = input(f"Marital Status {list(marital_encoder.classes_)}: ")
    vehicle = input(f"Vehicle Type {list(vehicle_encoder.classes_)}: ")
    education = input(f"Education {list(education_encoder.classes_)}: ")
    return {'Income': income, 'Marital Status': marital, 'Vehicle Class': vehicle, 'Education': education}


In [37]:
def process_user_data(details):
    user_df = pd.DataFrame([details])
    try:
        user_df['Marital Status'] = marital_encoder.transform(user_df['Marital Status'])
        user_df['Vehicle Class'] = vehicle_encoder.transform(user_df['Vehicle Class'])
        user_df['Education'] = education_encoder.transform(user_df['Education'])
    except ValueError:
        print("Invalid input! Please select from the provided options.")
        return None
    user_df[['Customer Lifetime Value', 'Total Claim Amount']] = 0  # Default values
    user_df[numeric_cols] = num_scaler.transform(user_df[numeric_cols])
    return user_df

In [38]:
def suggest_policy(user_data):
    if user_data is None:
        return None
    test_data = [np.concatenate([user_data.iloc[0].values, policies.iloc[j][['target_value', 'target_income', 'target_claims', 'target_marital', 'target_vehicle', 'target_education']].values]) for j in range(len(policies))]
    scores = xgb_model.predict(xgb.DMatrix(np.array(test_data)))
    
    suggestions = policies[['policy_name', 'monthly_cost']].copy()
    suggestions['match_score'] = scores
    suggestions['profit_factor'] = suggestions['monthly_cost'] / suggestions['monthly_cost'].max()
    suggestions['final_score'] = 0.7 * suggestions['match_score'] + 0.3 * suggestions['profit_factor']
    best_policy = suggestions.sort_values(by='final_score', ascending=False).iloc[0]
    
    reason = "Fits your profile."
    if best_policy['policy_name'] in ['Term Life', 'Term Insurance 1', 'Term Insurance 2'] and user_data['Marital Status'].iloc[0] == 1:
        reason = "Ideal for your family."
    elif best_policy['policy_name'] in ['Car Insurance', 'Car Insurance 1', 'Car Insurance 2'] and user_data['Vehicle Class'].iloc[0] in [3, 4, 5]:
        reason = "Suits your vehicle."
    elif best_policy['policy_name'] == 'Health Plan' and user_data['Education'].iloc[0] >= 2:
        reason = "Matches your education level."
    
    return {
        'Policy': best_policy['policy_name'],
        'Cost': best_policy['monthly_cost'],
        'Score': round(best_policy['final_score'], 3),
        'Reason': reason
    }


In [39]:
print("Welcome to Policy Recommender\n")
user_details = get_user_details()
processed_data = process_user_data(user_details)
if processed_data is not None:
    suggestion = suggest_policy(processed_data)
    if suggestion:
        print(f"\nRecommended Policy: {suggestion['Policy']}")
        print(f"Monthly Cost: ${suggestion['Cost']}")
        print(f"Confidence: {suggestion['Score']}")
        print(f"Why: {suggestion['Reason']}")

Welcome to Policy Recommender

Enter your info for a policy recommendation:


Annual Income (e.g., 50000):  5000
Marital Status ['Divorced', 'Married', 'Single']:  Single
Vehicle Type ['Four-Door Car', 'Luxury Car', 'Luxury SUV', 'SUV', 'Sports Car', 'Two-Door Car']:  SUV
Education ['Bachelor', 'College', 'Doctor', 'High School or Below', 'Master']:  Master



Recommended Policy: Term Insurance 2
Monthly Cost: $700
Confidence: 0.389
Why: Fits your profile.
